In [ ]:
import pandas as pd
from pytorch_forecasting import TimeSeriesDataSet, DeepAR, Baseline, Trainer
from pytorch_lightning import seed_everything
import torch

# 读取并整理数据
df = pd.read_csv("your_transactions.csv")
df['date'] = pd.to_datetime(df['date'])
df['time_idx'] = df.groupby("advisor_id")['date'].rank(method="dense").astype(int)  # 时间索引
df['group_id'] = df['advisor_id']  # 用来分组预测

# 可选静态分类变量
df['province'] = df['province'].astype(str)
df['fund_id'] = df['fund_id'].astype(str)

# 时间序列数据集
max_encoder_length = 60
max_prediction_length = 30

training = TimeSeriesDataSet(
    df,
    time_idx="time_idx",
    target="purchase_amount",
    group_ids=["group_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["group_id", "province", "fund_id"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["purchase_amount"]
)


In [ ]:
train_dataloader = training.to_dataloader(train=True, batch_size=64, num_workers=0)


In [ ]:
from pytorch_forecasting.models import DeepAR
from pytorch_lightning import Trainer

model = DeepAR.from_dataset(training, learning_rate=1e-3)

trainer = Trainer(max_epochs=10, gradient_clip_val=0.1)
trainer.fit(model, train_dataloaders=train_dataloader)


In [ ]:
raw_predictions, x = model.predict(train_dataloader, mode="raw", return_x=True)

# 可视化第一个序列预测
model.plot_prediction(x, raw_predictions, idx=0)



In `PyTorch Forecasting`:

- Embedding layers are created automatically for categorical features.
- Uses LSTM to capture temporal dependencies.
- Outputs a distribution for each future time step (not just a number).

---

## 🎯 Model Inputs

### 1. **Static Categorical Features**

These don't change over time:
- `advisor_id` (used as `group_id`)
- `province`
- `fund_id`

➡️ These are embedded (via `nn.Embedding`) and **repeated across time steps**.

### 2. **Time-varying Features**

#### Known (future known):
- `time_idx` (mandatory)
- `day_of_week`, `month`, `is_holiday` (optional)

#### Unknown (target):
- `purchase_amount` – only available in history during training

### 3. **Group IDs**

Each time series (e.g., an advisor's data) belongs to a `group_id`.

---

## 🔮 Forecasting Process

### ✅ During Training:
- Past 60 days of `purchase_amount` are encoded by LSTM.
- Model learns to output:
  - μ (mean)
  - σ (standard deviation)
- Optimizes **negative log likelihood (NLL)**.

### ✅ During Prediction:
- Encodes past 60 steps.
- **Autoregressively** predicts next 30:
  - Day t prediction becomes input for day t+1
- Produces:
  - Mean forecast
  - Distribution (μ, σ)
  - Sampled paths (e.g., 1000 futures)

---

## 📈 Output Types

- `predict()` returns mean by default.
- Use `mode='raw'` to get:
  - `μ`: mean
  - `σ`: std
  - `samples`: sampled paths from the distribution

---

## ✅ Why DeepAR Works Well for You

| Your Problem | DeepAR Advantage |
|--------------|------------------|
| Multiple advisors | ✔️ Handles multi-entity sequences via `group_id` |
| Sparse individual data | ✔️ Learns shared behavior via embeddings |
| Forecast 30 days | ✔️ Native multi-step output |
| Static variables (province, fund) | ✔️ Embedded for entity context |
| Want uncertainty, not just point | ✔️ Predicts distributions (not just values) |

---

## 🧠 Summary

> **DeepAR** is a neural probabilistic time series model that learns from multiple related sequences using LSTM and embeddings, outputs full predictive distributions, and supports multi-step forecasts.

---

Let me know if you'd like to:
- Visualize prediction distributions
- Compare DeepAR to other models
- Extract forecasts for a specific advisor
